# Autonomous driving - Traffic Light detection

In [1]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes, convert
from YAD2K.yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

%matplotlib inline

boxPath = './test/frameAnnotationsBOX.csv'
bulbPath = './test/frameAnnotationsBULB.csv'
img_path = './test/frames/'

C:\Program Files\Anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = .6):
    """ Filters YOLO boxes by thresholding on object and class confidence"""
    # Compute box scores
    box_scores = box_confidence * box_class_probs
    
    # Find the box_classes, keep track of the corresponding score
    box_classes = K.argmax(box_scores, axis=-1)
    box_class_scores = K.max(box_scores, axis=-1)
    
    # Create a filtering mask 
    filtering_mask = K.greater_equal(box_class_scores, threshold)
    
    # Apply the mask to scores, boxes and classes
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes,filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)
    
    return scores, boxes, classes

In [3]:
def iou(box1, box2):
    """ Intersection over union (IoU) between box1 and box2 """
    # Calculate (y1, x1, y2, x2)
    xi1 = np.maximum(box1[0], box2[0])
    yi1 = np.maximum(box1[1], box2[1])
    xi2 = np.minimum(box1[2], box2[2])
    yi2 = np.minimum(box1[3], box2[3])
    inter_area = max(xi2 - xi1, 0) * max(yi2 - yi1, 0)
    
    # Calculate Union area
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1]) 
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = (box1_area + box2_area) - inter_area
    
    # Compute IoU
    iou = inter_area / union_area
    
    return iou

In [4]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes=10, iou_threshold=0.5):
    """ apply non-max suppression to boxes"""
    
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
    
    # get list of indices corresponding to boxes we keep
    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes_tensor, iou_threshold=iou_threshold)
    
    # select only nms_indices from scores, boxes, classes
    scores = K.gather(scores, nms_indices)
    boxes = K.gather(boxes, nms_indices)
    classes = K.gather(classes, nms_indices)
    
    return scores, boxes, classes

In [5]:
def yolo_eval(yolo_outputs, image_shape = (960., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    """ Converts the outputs of YOLO encoding """
    # retrieve outputs of YOLO model
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs
    
    # Convert boxes to be ready for filtering functions
    boxes = yolo_boxes_to_corners(box_xy, box_wh)
    
    # Perform score filtering
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold=score_threshold)
    
    # Scale boxes back to original image shape
    boxes = scale_boxes(boxes, image_shape)
    
    # Perform non-max suppression
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes=max_boxes, iou_threshold=iou_threshold)
    
    return scores, boxes, classes

# Test YOLO pretrained model

In [6]:
sess = K.get_session()

# define classes, anchors, image shape
class_names = read_classes("./YAD2K/model_data/classes.txt")
anchors = read_anchors("./YAD2K/model_data/yolo_anchors.txt")
image_shape = (960., 1280.)

# load pretrained model # model.save() instead of save_weights
yolo_model = load_model("./YAD2K/model_data/trained.h5")
yolo_model.summary()

# Convert model output to usable bounding box tensors
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

# Filter boxes
scores, boxes, classes = yolo_eval(yolo_outputs, image_shape)

C:\Program Files\Anaconda3\envs\tf\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 608, 608, 3)  0                                            
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 608, 608, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_23 (BatchNo (None, 608, 608, 32) 128         conv2d_25[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_23 (LeakyReLU)      (None, 608, 608, 32) 0           batch_normalization_23[0][0]     
__________________________________________________________________________________________________
max_poolin

In [7]:
def predict(sess, image_file, img_path=img_path):
    """ Runs the graph to predict boxes"""
    # Preproccess image
    image, image_data = preprocess_image(img_path + image_file, model_image_size=(608, 608))
    
    # run session with tensors
    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes], feed_dict = {yolo_model.input: image_data, K.learning_phase(): 0})
    
    # Print predictions info
    print('Found {} boxes for {}'.format(len(out_boxes), image_file))
    # Generate colors for drawing bounding boxes.
    colors = generate_colors(class_names)
    # Draw bounding boxes on the image file
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    # Save the predicted bounding box on the image
    image.save(os.path.join("./test/out", image_file), quality=90)
    # Display the results in the notebook
    #output_image = scipy.misc.imread(os.path.join("out", image_file))
    #imshow(output_image)
    
    return out_scores, out_boxes, out_classes

In [8]:
# Count the number of correct predictions 
def n_correct(box, y_true, iou_threshold=0.5):
    n_c = 0
    # check against every box
    for index, row in y_true.iterrows():
        if iou(box, (row['Upper left corner Y'], row['Upper left corner X'],
                      row['Lower right corner Y'], row['Lower right corner X'])) > iou_threshold:
            n_c += 1
    return n_c

In [9]:
def get_precision(sess, img, boxPath, bulbPath, n_classes=7):
    # Concat the Boxes and Bulbs data 
    boxes = convert(boxPath, bulbPath)
    out_scores, out_boxes, out_classes = predict(sess, img)
    y_true = boxes[boxes['Filename'].str.match(img)]
    classes = np.zeros([n_classes])
    true_classes = np.zeros([n_classes])
    
    # get the number of correct predictions for each class
    for i, c in enumerate(out_classes):
        classes[c] += n_correct(out_boxes[i], y_true)
   
    # get the number of true boxes per class
    for index, row in y_true.iterrows():
        true_classes[row['Annotation tag']] += 1
    
    print(out_classes)
    print("---->", int(np.sum(true_classes)), " boxes in total")
    print(true_classes)
    precision = np.divide(classes, true_classes)
    precision[np.isnan(precision)] = 0
    precision[np.isinf(precision)] = 0
    precision[np.isneginf(precision)] = 0
    return precision, true_classes

In [10]:
def avg_precision(sess, boxPath, bulbPath, img_path, n_classes=7):
    boxes = convert(boxPath, bulbPath)
    precision = np.zeros([n_classes])
    total = np.zeros([n_classes])
    for img in os.listdir(img_path):
        # calculate the precision for each image
        precision_img, true_classes = get_precision(sess, img, boxPath, bulbPath)
        precision += precision_img
        for i in range(len(total)):
            # if class is in the picture count up
            if true_classes[i] > 0.:
                total[i] += 1
    avg_precision = np.divide(precision, total)
    avg_precision[np.isnan(avg_precision)] = 0
    avg_precision[np.isinf(avg_precision)] = 0
    avg_precision[np.isneginf(avg_precision)] = 0
    return avg_precision, total

In [11]:
def mean_average_precision(sess, boxPath, bulbPath, img_path, n_classes=7):
    avg_pr, total = avg_precision(sess, boxPath, bulbPath, img_path)
    mAP = np.divide(np.sum(avg_pr), len(total[total > 0]))
    print("mAP Score: ", mAP)
    print(avg_pr)
    for i in range(len(avg_pr)):
        if avg_pr[i] > 0.:
            print("Average Precision for class", i, ": ", avg_pr[i])
    return mAP, avg_pr

In [15]:
mAP, _ = mean_average_precision(sess, boxPath, bulbPath, img_path)

Found 0 boxes for daySequence1--00000.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]


C:\Program Files\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


Found 0 boxes for daySequence1--00001.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--00002.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--00003.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--00004.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--00005.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--00006.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--00007.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--00008.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--00009.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--00010.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--00011.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0

Found 1 boxes for daySequence1--00090.jpg
[5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for daySequence1--00091.jpg
[5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for daySequence1--00092.jpg
[5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for daySequence1--00093.jpg
[5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for daySequence1--00094.jpg
[5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--00095.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for daySequence1--00096.jpg
[5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for daySequence1--00097.jpg
[5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for daySequence1--00098.jpg
[5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for daySequence1--00099.jpg
[5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for daySequence1--00100.jpg
[5]
----> 0  boxes in total
[

Found 1 boxes for daySequence1--00179.jpg
[5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for daySequence1--00180.jpg
[5 5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--00181.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for daySequence1--00182.jpg
[5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--00183.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for daySequence1--00184.jpg
[5 5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for daySequence1--00185.jpg
[5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for daySequence1--00186.jpg
[5 5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for daySequence1--00187.jpg
[5 5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for daySequence1--00188.jpg
[5 5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for daySequence1--00189.jpg
[5 5]
----> 0  boxes

...

Found 0 boxes for daySequence1--03713.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03714.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03715.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03716.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03717.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03718.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03719.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03720.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03721.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03722.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03723.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0

Found 0 boxes for daySequence1--03803.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03804.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03805.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03806.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03807.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03808.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03809.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03810.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03811.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03812.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 0 boxes for daySequence1--03813.jpg
[]
----> 0  boxes in total
[0. 0. 0. 0

Found 3 boxes for daySequence1--03892.jpg
[0 0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for daySequence1--03893.jpg
[0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for daySequence1--03894.jpg
[0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for daySequence1--03895.jpg
[0 0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for daySequence1--03896.jpg
[0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for daySequence1--03897.jpg
[0 5]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for daySequence1--03898.jpg
[0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for daySequence1--03899.jpg
[0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for daySequence1--03900.jpg
[0 0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 1 boxes for daySequence1--03901.jpg
[0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for daySequence1--03902.jpg
[0 0 0]


Found 5 boxes for daySequence1--03978.jpg
[0 0 0 0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 5 boxes for daySequence1--03979.jpg
[0 0 0 0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 6 boxes for daySequence1--03980.jpg
[0 0 0 0 0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 5 boxes for daySequence1--03981.jpg
[0 0 0 0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 4 boxes for daySequence1--03982.jpg
[0 0 0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for daySequence1--03983.jpg
[0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for daySequence1--03984.jpg
[0 0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for daySequence1--03985.jpg
[0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for daySequence1--03986.jpg
[0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 2 boxes for daySequence1--03987.jpg
[0 0]
----> 0  boxes in total
[0. 0. 0. 0. 0. 0. 0.]
Found 3 boxes for 

C:\Program Files\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]
C:\Program Files\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
